In [1]:
%load_ext autoreload
%autoreload 2
# You can load your python module as this:
import modelproject.example

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
fafoihan

NameError: name 'fafoihan' is not defined

In [1]:
fjfpoaj


NameError: name 'fjfpoaj' is not defined